In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from pandas.api.types import CategoricalDtype

from sklearn.impute import SimpleImputer
from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from pathlib import Path
from scipy.stats import skew 


In [ ]:
data_dir = './input/'
# data_dir = Path("../input/house-prices-advanced-regression-techniques/")
X_train = pd.read_csv(data_dir + 'train.csv', index_col="id")
X_test = pd.read_csv(data_dir + 'test.csv', index_col="id")

## EDA (Exploratory Data Analysis)

hasil :
### missing values :
Missing values:
- Guest_Popularity_percentage    (146030) (19%)
- Episode_Length_minutes          (87093) (12%)
- Number_of_Ads                       (1)

In [ ]:
def explore_data(df):
    print(f"DataFrame shape: {df.shape}")
    print(f"DataFrame columns: {df.columns.tolist()}")
    print(f"DataFrame info:\n{df.info()}")
    print(f"DataFrame description:\n{df.describe(include='all')}")
    print(f"Missing values:\n{df.isnull().sum().sort_values(ascending=False).head(20)}")
    print(f"Duplicate rows: {df.duplicated().sum()}")

In [ ]:
explore_data(X_train)

In [ ]:
X_train

## Preprocessing

beberapa funtion untuk preprocessing :

- clean() - potensi regex
- encode() 
- impute() - masih bisa dikembangin
- outlier_check()


In [ ]:
# disini bisa buat fitur episode_number
def clean(df):

    df['Episode_Number'] = df['Episode_Title'].str.extract(r'(\d+)').astype(float)
    df = df.drop('Episode_Title', axis=1)

    return df

    

In [ ]:
def encode(df):
    # The nominative (unordered) categorical features
    features_nom = [
    'Podcast_Name',
    'Genre',
    'Publication_Day',
    ]
    
    features_ord = ['Episode_Sentiment', 'Publication_Time']

    ordered_levels = {
        'Episode_Sentiment': ['Negative', 'Neutral', 'Positive'],
        'Publication_Time': ['Morning', 'Afternoon', 'Evening', 'Night']
    }

    # Add a None level for missing values
    ordered_levels = {key: ["None"] + value for key, value in
                  ordered_levels.items()}

    
    # Nominal categories
    for name in features_nom:
        df[name] = df[name].astype("category")
        # Add a None category for missing values
        if "None" not in df[name].cat.categories:
            df[name] = df[name].cat.add_categories("None")
    # Ordinal categories
    for name, levels in ordered_levels.items():
        df[name] = df[name].astype(CategoricalDtype(levels,
                                                    ordered=True))
    
    return df
    

In [ ]:
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna("None")
    return df

def impute_upgraded(df):
    for name in df.select_dtypes("number").columns:
        df[name] = df[name].fillna(df[name].median())
    for name in df.select_dtypes("category").columns:
        df[name] = df[name].fillna(df[name].mode().iloc[0])  # mode bisa punya banyak nilai
    return df

def impute_fillna_mean(df):
    # numerical features
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(df[name].mean())
    
    # categorical features
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna(df[name].mode().iloc[0])

    return df


def impute_fillna_median(df):
    # numerical features
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(df[name].median())
 
    # categorical features
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna(df[name].mode().iloc[0])

    return df


def impute_simple_mean(df):
    #numerical features
    num_features = df.select_dtypes(include=['int64', 'float64']).columns
    num_imputer = SimpleImputer(strategy='mean')
    df[num_features] = num_imputer.fit_transform(df[num_features])

    #categorical features
    cat_features = df.select_dtypes(include=['object', 'category']).columns
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df[cat_features] = cat_imputer.fit_transform(df[cat_features])

    for col in cat_features:
        df[col] = df[col].astype('category').cat.codes

    return df   

def impute_simple_median(df):
    #numerical features
    num_features = df.select_dtypes(include=['int64', 'float64']).columns
    imputer = SimpleImputer(strategy='median')
    df[num_features] = imputer.fit_transform(df[num_features])

    #categorical features
    cat_features = df.select_dtypes(include=['object', 'category']).columns
    imputer = SimpleImputer(strategy='most_frequent')
    df[cat_features] = imputer.fit_transform(df[cat_features])

    for col in cat_features:
        df[col] = df[col].astype('category').cat.codes
    return df 


In [ ]:
def outlier_check(df, feature, log=False, return_filtered=False, q_low=0.01, q_high=0.99, plot=True):

    # Menampilkan grafik distribusi fitur jika plot=True
    if plot:
        print(f"📊 Distribusi Fitur: {feature}")
        plt.figure(figsize=(12, 4))
        sns.histplot(df[feature], bins=100, kde=True, color='skyblue')
        plt.title(f'Distribusi asli: {feature}')
        plt.xlabel(feature)
        plt.ylabel('Frekuensi')
        plt.show()

        if log:
            plt.figure(figsize=(12, 4))
            sns.histplot(np.log1p(df[feature]), bins=100, kde=True, color='salmon')
            plt.title(f'Distribusi log(1 + {feature})')
            plt.xlabel(f"log(1 + {feature})")
            plt.ylabel('Frekuensi')
            plt.show()

    # Menghitung quantile
    ql = df[feature].quantile(q_low)
    qh = df[feature].quantile(q_high)

    if plot:
        print(f"🔍 Quantile batas:")
        print(f" - {int(q_low*100)}th percentile: {ql:.2f}")
        print(f" - {int(q_high*100)}th percentile: {qh:.2f}")

    # Filter data berdasarkan quantile
    df_filtered = df[(df[feature] >= ql) & (df[feature] <= qh)]
    if plot:
        print(f"✅ Jumlah data setelah filter: {df_filtered.shape[0]} dari {df.shape[0]} ({100 * df_filtered.shape[0] / df.shape[0]:.2f}%)")
    
    print("Sisa data:", df_filtered.shape)
    
    # Mengembalikan hasil filter jika return_filtered=True
    if return_filtered:
        return df_filtered  # Hanya mengembalikan data yang sudah difilter


In [ ]:
df_noOutlier = outlier_check(X_train, 'Episode_Length_minutes', log=True, return_filtered=True)

In [ ]:
def skewness():
    df_train = pd.read_csv(data_dir + 'train.csv', index_col="id")
    return df_train

numerical_features = [
        "Episode_Length_minutes",
        "Host_Popularity_percentage",
        "Guest_Popularity_percentage",
        "Number_of_Ads",
        "Listening_Time_minutes",
    ]

df_train = skewness()

for feature in numerical_features:
    plt.figure(figsize=(8, 4))

    # Histogram with KDE (Kernel Density Estimate)
    plt.subplot(1, 2, 1)
    sns.histplot(df_train[feature], kde=True, bins=30)
    plt.title(f"Histogram of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Frequency")

    # Box plot to identify outliers
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_train[feature])
    plt.title(f"Box Plot of {feature}")

    plt.tight_layout()
    plt.show()

    # Print additional statistics
    print(f"\nStatistics for {feature}:")
    print(f"Skewness: {df_train[feature].skew():.2f}")

## Load Data

- load_data()
- panggil load_data()

In [ ]:
def load_data():
    #Read data
    data_dir = 'input/'
    # data_dir = Path("../input/house-prices-advanced-regression-techniques/")

    df_train = pd.read_csv(data_dir + 'train.csv', index_col="id")
    df_test = pd.read_csv(data_dir + 'test.csv', index_col="id")

    #Merge the splits so we can preprocess them together
    df = pd.concat([df_train, df_test])
    #Preprocessing
    df = clean(df)
    df = encode(df)
    df = impute_upgraded(df)

    # df['is_weekend'] = df['Publication_Day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)


    #reform splits
    df_train = df.loc[df_train.index, :]
    df_test = df.loc[df_test.index, :]

    # df_train = outlier_check(df_train, 'Episode_Length_minutes', log=True, return_filtered=True, plot=False)


    return df_train, df_test

def load_data_for_baseLine():
    data_dir = 'input/'

    df_train = pd.read_csv(data_dir + 'train.csv', index_col="id")
    df_test = pd.read_csv(data_dir + 'test.csv', index_col="id")

    # Simpan panjang data asli untuk df_train dan df_test
    train_len = len(df_train)
    test_len = len(df_test)

    # Gabungkan df_train dan df_test
    df = pd.concat([df_train, df_test])

    # Preprocessing
    df = clean(df)
    df = encode(df)
    #df = impute_fillna_median(df)
    #df = impute_upgraded(df)
    df = impute(df)

    # df['is_weekend'] = df['Publication_Day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

    
    # df['Number_of_Ads_log'] = np.log1p(df['Number_of_Ads'])
    # df['Guest_Popularity_percentage_log'] = np.log1p(df['Guest_Popularity_percentage'])
    # df['Host_Popularity_percentage_log'] = np.log1p(df['Host_Popularity_percentage'])
    # df['Episode_Length_minutes_log'] = np.log1p(df['Episode_Length_minutes']) 
    
    # df = df.drop(columns=['Number_of_Ads'])
    # df = df.drop(columns=['Guest_Popularity_percentage'])
    # df = df.drop(columns=['Host_Popularity_percentage'])
    # df = df.drop(columns=['Episode_Length_minutes'])
    

    # Pisahkan kembali df_train dan df_test berdasarkan panjang data asli
    df_train = df.iloc[:train_len, :]
    df_test = df.iloc[train_len:train_len + test_len, :]

    # df_train = outlier_check(df_train, 'Episode_Length_minutes', log=True, return_filtered=True, plot=False)



    return df_train, df_test

## Base line

- score_dataset()
- cek liat score

Baseline : 13.20727 RMSE

In [ ]:
df_train, df_test = load_data_for_baseLine()

In [ ]:
df_train

In [ ]:
xgb_params = {
    'random_state': 0,
    'n_estimators': 565,
    'max_depth': 14,
    'learning_rate': 0.04222221,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,    
    'tree_method':'hist', 
    # 'tree_method':'gpu_hist', 
    'n_jobs': -1  
}

#xgbmodel
xgb_model = XGBRegressor(
    random_state = 0,
    n_estimators = 565,
    max_depth= 14,
    learning_rate = 0.04222221,
    subsample= 0.8,
    colsample_bytree = 0.8,   
    n_jobs= -1 
)

#lgbmodel
lgbm_model = LGBMRegressor(
        random_state = 0,
        n_iter=1000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        verbosity=-1,
        max_bin=1024,
)

In [ ]:
def score_dataset(X, y, model=None):
    # Label encoding for categoricals
    for colname in X.select_dtypes("category"):
        X[colname] = X[colname].cat.codes
    
    # Cross-validation pakai RMSE
    score = cross_val_score(
        model, X, y, 
        cv=5,
        scoring="neg_mean_squared_error"
    )

    score = -1 * score.mean()
    score = np.sqrt(score)  #matriknya make rmse
    return score

In [ ]:

X = df_train.copy()
y = X.pop("Listening_Time_minutes")

baseline_score = score_dataset(X, y, XGBRegressor(random_state=0))
print(f"Baseline score: {baseline_score:.5f} RMSE")


In [ ]:
#Baseline With KFold

from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def score_dataset_Kfold(X,y, X_test):

    X = X.copy()
    X_test = X_test.copy()
    
    for colname in X.select_dtypes("category"):
        X[colname] = X[colname].cat.codes
        
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    test_preds = np.zeros(len(X_test))


    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}/{n_splits}...")    
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]   
        model = XGBRegressor()
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)    
        val_pred = model.predict(X_val)
        score = rmse(y_val, val_pred)
        scores.append(score)
        test_preds += model.predict(X_test) / n_splits      
        print(f"Fold {fold + 1} RMSE: {score:.4f}")

    print(f'Optimized Cross-validated RMSE score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'Max RMSE score: {np.max(scores):.5f}')
    print(f'Min RMSE score: {np.min(scores):.5f}')

    return test_preds

In [ ]:
X = df_train.copy()
y = X.pop("Listening_Time_minutes")
X_test = df_test.copy()
X_test = X_test.drop(columns=["Listening_Time_minutes"], errors='ignore')  # drop kalau ada

pred =score_dataset_Kfold(X, y, X_test)

## Feature Engineering
Hasil experiment :

### Improve
- cluster_labels : naik dari 13.20727 jadi 13.20711 RMSE
- Feature TopFeaturesCombined : Improvement: 0.179172
- Feature GuestImpact : Improvement: 0.149887
- Feature HostImpact : Improvement: 0.146457
- Feature ContentDensity : Improvement: 0.138797
- Impute median : naik dari 13.20727 jadi 13.03053 RMSE 
- Model Tuning : naik dari 13.20727 jadi 12.82695 RMSE
- simpleImputer : naik dari jadi 13.20727 13.03053 RMSE


### Degrade
- pca_components : turun dari 13.20727 jadi 13.226382669875116
- Feature PopularityScore : Improvement: -0.006710
- Feature PopularityScore : Improvement: -0.006710
- Feature EngagementFactor : Improvement: -0.004424
- Feature PodcastMomentum : Improvement: -0.005147
- Feature TimeDayInteraction : Improvement: -0.001016
- remove outlier malah turun

### No Change
- NormalizedEpisodeNumber : Improvement: 0.000000
- Skew fix: Number_of_Ads, Guest_Popularity_percentage, Host_Popularity_percentage, Episode_Length_minutes : Improvement : 0

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
X = df_train.copy()
y = X.pop("Listening_Time_minutes")

In [ ]:
mi_scores = make_mi_scores(X, y)
print("Mutual Information Scores:")
mi_scores

In [ ]:
def label_encode(df):
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    return X

### K-means clustering


In [ ]:
#K-means clustering

cluster_features = [
    "Episode_Length_minutes",
    "Host_Popularity_percentage",
    "Guest_Popularity_percentage",
    "Episode_Number"
]


def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)               #TUNING
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new

def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)                          #TUNING
    X_cd = kmeans.fit_transform(X_scaled)
    # Label features and join to dataset
    X_cd = pd.DataFrame(
        X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])]
    )
    return X_cd

In [ ]:
df_cluster_labels= cluster_labels(X, cluster_features, n_clusters=20)
df_cluster_distance = cluster_distance(X, cluster_features, n_clusters=20)


In [ ]:
df_cluster = pd.concat([df_cluster_labels, df_cluster_distance], axis=1)
df_cluster

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scaling ulang agar cocok untuk PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[cluster_features])

# PCA untuk reduksi ke 2 dimensi
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Gabungkan hasil PCA dan label cluster
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['Cluster'] = df_cluster['Cluster']

# Plot
plt.figure(figsize=(10, 6))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Cluster'], cmap='tab20', alpha=0.7)
plt.title('Visualisasi Cluster dengan PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.show()

### PCA (Principal Component Analysis)

- pca_components()
- pca_inspired()

In [ ]:
#Principal Component Analysis

def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings

def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs


#gabung-gabungin feature dari analysis PCA
def pca_inspired(df):
    X = pd.DataFrame()
    X["TopFeaturesCombined"] = df.Episode_Length_minutes * df.Host_Popularity_percentage * df.Episode_Number
    X["GuestImpact"] = df.Guest_Popularity_percentage * df.Episode_Length_minutes
    X["HostImpact"] = df.Host_Popularity_percentage * df.Episode_Length_minutes
    X["ContentDensity"] = df.Episode_Length_minutes / (df.Number_of_Ads + 1)
    return X

def pca_components(df, features):
    X = df.loc[:, features]
    _, X_pca, _ = apply_pca(X)
    return X_pca

pca_features = [
    "Episode_Length_minutes",         # MI tinggi
    "Host_Popularity_percentage",
    "Guest_Popularity_percentage",
    "Episode_Number",
    "Number_of_Ads"
]

def create_features_categorical(df):
    df['is_weekend'] = df['Publication_Day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

    return df


In [ ]:
X_pca = pca_components(X, pca_features)
X_pca

In [ ]:
df_pca = X.loc[:, pca_features]
pca, X_pca, loadings = apply_pca(df_pca)
plot = plot_variance(pca)
plot

#### Mencari Feature terbaik dari untuk pca_inspired()

Baseline XGBoost MSRE: 13.207268 ± 1.076183
- With PopularityScore: MSRE = 13.213978 ± 1.090161 (Improvement: -0.006710)
- With HostImpact: MSRE = 13.060811 ± 0.914555 (Improvement: 0.146457)
- With GuestImpact: MSRE = 13.057381 ± 1.046029 (Improvement: 0.149887)
- With NormalizedEpisodeNumber: MSRE = 13.207268 ± 1.076183 (Improvement: 0.000000)
- With EngagementFactor: MSRE = 13.211691 ± 1.118318 (Improvement: -0.004424)
- With PodcastMomentum: MSRE = 13.212414 ± 1.069311 (Improvement: -0.005147)
- With TimeDayInteraction: MSRE = 13.208284 ± 1.060503 (Improvement: -0.001016)
- With ContentDensity: MSRE = 13.068470 ± 1.030627 (Improvement: 0.138797)
- With TopFeaturesCombined: MSRE = 13.028096 ± 0.956762 (Improvement: 0.179172)


In [ ]:

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer


# Fungsi custom untuk menghitung MSRE (Mean Squared Relative Error)
def msre(y_true, y_pred):
    """
    Mean Squared Relative Error: mean((y_true - y_pred)^2 / y_true^2)
    """
    # Hindari pembagian dengan 0
    mask = y_true != 0
    y_true_safe = y_true[mask]
    y_pred_safe = y_pred[mask]
    
    if len(y_true_safe) == 0:
        return 0.0
    
    # Hitung MSRE
    relative_errors = ((y_true_safe - y_pred_safe) ** 2) / (y_true_safe ** 2)
    return np.mean(relative_errors)

# Buat scorer untuk digunakan dalam cross_val_score
msre_scorer = make_scorer(msre, greater_is_better=False)

def evaluate_xgb_features(df, target_col, features_list, params=None):

    results = {}    
    
    # Baseline model dengan fitur asli
    X_baseline = df.drop(target_col, axis=1)
    baseline_features = X_baseline.columns.tolist()
    y = df[target_col]
    
    # Evaluasi baseline dengan XGBoost
    model = XGBRegressor()
    baseline_scores = cross_val_score(model, X_baseline, y, cv=5, scoring='neg_mean_squared_error')
    # Negasikan karena make_scorer menghasilkan -MSRE untuk greater_is_better=False
    baseline_msre = -np.mean(baseline_scores)
    baseline_msre = np.sqrt(baseline_msre)
    
    results['baseline'] = {
        'msre': baseline_msre,
        'std': np.std(-baseline_scores),
        'features_used': baseline_features
    }
    
    print(f"Baseline XGBoost MSRE: {baseline_msre:.6f} ± {np.std(-baseline_scores):.6f}")
    
    # Buat dan evaluasi setiap fitur baru secara individual
    for feature_name, feature_func in features_list.items():
        try:
            # Buat fitur baru
            new_feature = feature_func(df)
            
            if isinstance(new_feature, pd.Series):
                new_feature = pd.DataFrame({feature_name: new_feature})
            
            # Gabungkan dengan fitur baseline
            X_combined = pd.concat([X_baseline, new_feature], axis=1)
            
            # Evaluasi dengan XGBoost dan cross-validation
            model = XGBRegressor()
            scores = cross_val_score(model, X_combined, y, cv=5, scoring='neg_mean_squared_error')
            msre_value = -np.mean(scores)  # Negasikan untuk mendapatkan actual MSRE
            msre_value = np.sqrt(msre_value) 
            
            # Simpan hasil
            results[feature_name] = {
                'msre': msre_value,
                'std': np.std(-scores),
                'improvement': baseline_msre - msre_value,  # Positive means better (lower MSRE)
                'features_used': X_combined.columns.tolist()
            }
            
            print(f"With {feature_name}: MSRE = {msre_value:.6f} ± {np.std(-scores):.6f} " +
                  f"(Improvement: {baseline_msre - msre_value:.6f})")
            
        except Exception as e:
            print(f"Error evaluating {feature_name}: {str(e)}")
    
    return results

def plot_xgb_feature_results(results):
    """
    Visualisasi hasil evaluasi fitur
    """
    features = list(results.keys())
    msre_values = [results[f]['msre'] for f in features]
    improvements = [results[f]['improvement'] if f != 'baseline' else 0 for f in features]
    
    # Sort by improvement (descending)
    sorted_indices = np.argsort([-imp for imp in improvements])
    sorted_features = [features[i] for i in sorted_indices]
    sorted_improvements = [improvements[i] for i in sorted_indices]
    
    plt.figure(figsize=(12, 8))
    colors = ['green' if imp > 0 else 'red' for imp in sorted_improvements]
    bars = plt.barh(sorted_features, sorted_improvements, color=colors)
    
    plt.xlabel('MSE Improvement (Positive is better)')
    plt.title('Feature Importance by MSE Improvement')
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
    
    # Tambahkan nilai di setiap bar
    for i, bar in enumerate(bars):
        value = sorted_improvements[i]
        plt.text(value + 0.0001 if value >= 0 else value - 0.005, 
                 bar.get_y() + bar.get_height()/2, 
                 f'{value:.6f}', 
                 va='center')
    
    plt.tight_layout()
    
    return plt


# Definisikan fitur yang akan diuji
pca_inspired_features = {
    "PopularityScore": lambda df: df.Host_Popularity_percentage * df.Guest_Popularity_percentage,
    
    "HostImpact": lambda df: df.Host_Popularity_percentage * df.Episode_Length_minutes,
    
    "GuestImpact": lambda df: df.Guest_Popularity_percentage * df.Episode_Length_minutes,
    
    "NormalizedEpisodeNumber": lambda df: df.Episode_Number / df.groupby('Podcast_Name')['Episode_Number'].transform('max'),
    
    "EngagementFactor": lambda df: df.Host_Popularity_percentage + df.Guest_Popularity_percentage + (df.Episode_Sentiment * 100),
    
    "PodcastMomentum": lambda df: df.Episode_Number * df.Host_Popularity_percentage,
    
    "TimeDayInteraction": lambda df: df.Publication_Time + (df.Publication_Day * 24),
    
    "ContentDensity": lambda df: df.Episode_Length_minutes / (df.Number_of_Ads + 1),
    
    "TopFeaturesCombined": lambda df: df.Episode_Length_minutes * df.Host_Popularity_percentage * df.Episode_Number
}

In [ ]:

df_train, df_test = load_data_for_baseLine()
df_train = label_encode(df_train)
results = evaluate_xgb_features(df_train, 'Listening_Time_minutes', pca_inspired_features)
plot = plot_xgb_feature_results(results)
plot.show()

In [ ]:
def corrplot(df, method="pearson", annot=True, **kwargs):
    sns.clustermap(
        df.corr(method, numeric_only=True),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )


corrplot(df_train, annot=None)

## Train & Predict

In [ ]:
def create_features(df, df_test=None):
    X = df.copy()
    y = X.pop("Listening_Time_minutes")

    if df_test is not None:
        X_test = df_test.copy()
        X_test.pop("Listening_Time_minutes")
        X = pd.concat([X, X_test])

    X = create_features_categorical(X)
    X = X.join(cluster_labels(X, cluster_features, n_clusters=20))
    # X = X.join(pca_components(X, pca_features))
    X = X.join(pca_inspired(X))

    X = label_encode(X)
    
    # Reform splits
    if df_test is not None:
        X_test = X.loc[df_test.index, :]
        X.drop(df_test.index, inplace=True)

    
    if df_test is not None:
        return X, X_test
    else:
        return X


### Evaluasi hasil create features

In [ ]:
df_train, df_test = load_data()
X_train, X_test = create_features(df_train, df_test)
y_train = df_train.loc[:,'Listening_Time_minutes']


In [ ]:
X_train

In [ ]:
print(f"Final Score : {score_dataset(X_train, y_train, model=XGBRegressor(**xgb_params)):.5f} RMSE")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def score_dataset_Kfold(X,y, X_test,model=None):

    X = X.copy()
    X_test = X_test.copy()
    
    for colname in X.select_dtypes("category"):
        X[colname] = X[colname].cat.codes
        
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    test_preds = np.zeros(len(X_test))


    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}/{n_splits}...")    
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]   
        kmodel = model
        kmodel.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)    
        val_pred = kmodel.predict(X_val)
        score = rmse(y_val, val_pred)
        scores.append(score)
        test_preds += kmodel.predict(X_test) / n_splits      
        print(f"Fold {fold + 1} RMSE: {score:.4f}")

    print(f'Optimized Cross-validated RMSE score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'Max RMSE score: {np.max(scores):.5f}')
    print(f'Min RMSE score: {np.min(scores):.5f}')

    return test_preds

In [ ]:
print(f"Final Score (Random Forest): {score_dataset(X_train, y_train, model=RandomForestRegressor(random_state=0)):.5f} RMSE")

### Train model

In [ ]:
#without kfold

xgb = XGBRegressor(**xgb_params)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)


In [ ]:
#with kfold

# xgb = XGBRegressor(**xgb_params)
# y_pred =score_dataset_Kfold(X_train, y_train, X_test)

## Submission


In [ ]:
def make_submisson():
    output = pd.DataFrame({'id': X_test.index, 'Listening_Time_minutes': y_pred})
    output.to_csv('my_submission6.csv', index=False)
    print("Your submission was successfully saved!")

In [ ]:
# make_submisson()

## Next Upcoming Experiment


1. Manipulasi Text feature "podcast Name" dengan regex atau yg lain

2. Stacking model


3. Interaksi Genre x TimeOfDay → genre tertentu perform lebih bagus pagi/malam?

4. Coba k-means clustering untuk Podcast_Name → beri cluster ID sebagai feature

### Stack Model

In [ ]:
def score_stacking_Kfold(X, y, X_test, base_models, meta_model):
    """
    X: training features
    y: training targets
    X_test: test features
    base_models: list of models
    meta_model: model for stacking
    """
    X = X.copy()
    X_test = X_test.copy()

    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    test_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}/{n_splits}...")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # 1. Buat meta-features untuk training dan validasi
        X_train_meta = np.zeros((len(X_train), len(base_models)))
        X_val_meta = np.zeros((len(X_val), len(base_models)))
        X_test_meta = np.zeros((len(X_test), len(base_models)))

        for i, model in enumerate(base_models):
            model.fit(X_train, y_train)
            X_train_meta[:, i] = model.predict(X_train)
            X_val_meta[:, i] = model.predict(X_val)
            X_test_meta[:, i] += model.predict(X_test) / n_splits  # Rata-rata nanti

        # 2. Train meta-model
        meta_model.fit(X_train_meta, y_train)

        # 3. Predict dan hitung skor di validation
        val_pred = meta_model.predict(X_val_meta)
        score = rmse(y_val, val_pred)
        scores.append(score)

        # 4. Predict untuk test set
        test_preds += meta_model.predict(X_test_meta) / n_splits

        print(f"Fold {fold + 1} RMSE: {score:.4f}")

    print(f'Optimized Cross-validated RMSE score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'Max RMSE score: {np.max(scores):.5f}')
    print(f'Min RMSE score: {np.min(scores):.5f}')

    return test_preds


In [ ]:
def stacking_regression_with_rmse(base_models, meta_model, X_train, y_train, X_test, n_splits=5):
    """
    base_models: list of models
    meta_model: stacking model
    X_train: training data
    y_train: training target
    X_test: test data
    n_splits: number of splits
    """

    # Prepare arrays
    n_train, n_test = X_train.shape[0], X_test.shape[0]
    n_models = len(base_models)
    
    base_predictions_train = np.zeros((n_train, n_models))
    base_predictions_test = np.zeros((n_test, n_models))
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    
    for i, model in enumerate(base_models):
        test_fold_predictions = np.zeros((n_test, n_splits))

        for j, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

            model.fit(X_tr, y_tr)
            base_predictions_train[valid_idx, i] = model.predict(X_val)
            test_fold_predictions[:, j] = model.predict(X_test)

        base_predictions_test[:, i] = test_fold_predictions.mean(axis=1)

    # Train meta model
    meta_model.fit(base_predictions_train, y_train)
    
    # Predict on training set
    train_pred = meta_model.predict(base_predictions_train)
    final_prediction = meta_model.predict(base_predictions_test)

    # Evaluate RMSE
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(base_predictions_train)):
        X_val = base_predictions_train[valid_idx]
        y_val = y_train.iloc[valid_idx]
        val_pred = meta_model.predict(X_val)
        score = rmse(y_val, val_pred)
        scores.append(score)
        print(f"Fold {fold+1} RMSE: {score:.4f}")

    print(f'Optimized Cross-validated RMSE score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'Max RMSE score: {np.max(scores):.5f}')
    print(f'Min RMSE score: {np.min(scores):.5f}')

    return final_prediction

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

base_models = [
    xgb,
    RandomForestRegressor(random_state=0),
    lgbm_model
]

meta_model = Ridge(random_state=0)



In [ ]:
final_test_prediction = score_stacking_Kfold(
    X_train, 
    y_train, 
    X_test, 
    base_models, 
    meta_model)


In [ ]:
final_predictions_2 = stacking_regression_with_rmse(
    base_models, 
    meta_model, 
    X_train, 
    y_train, 
    X_test,
    n_splits=5
)


In [ ]:
print(final_predictions_2)

In [ ]:
def make_submisson():
    output = pd.DataFrame({'id': X_test.index, 'Listening_Time_minutes': final_predictions_2})
    output.to_csv('my_submission7.csv', index=False)
    print("Your submission was successfully saved!")

In [ ]:
make_submisson()